# 地方競馬（NAR）ハイパーパラメータ最適化 (09モデル)

Optunaを用いて、08モデルで使用した特徴量セットに対する最適なLightGBMパラメータを探索します。

In [1]:
import sys
import os
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from scipy.stats import spearmanr
from sklearn.metrics import mean_squared_error

# プロジェクトのsrcディレクトリをパスに追加
src_path = os.path.abspath(os.path.join(os.getcwd(), '../../src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from nar.loader import NarDataLoader
from nar.features import NarFeatureGenerator

/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py:18: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from optuna import progress_bar as pbar_module


In [2]:
loader = NarDataLoader()
raw_df = loader.load(limit=150000, region='south_kanto')

# 08モデルと同じ特徴量生成（1-5走前、改善ロジック込）
generator = NarFeatureGenerator(history_windows=[1, 2, 3, 4, 5])
df = generator.generate_features(raw_df)

df = df.dropna(subset=['rank']).copy()
df['date'] = pd.to_datetime(df['date'])

# 特徴量リスト
baseline_features = [
    'distance', 'venue', 'state', 'frame_number', 'horse_number', 'weight', 'impost',
    'jockey_win_rate', 'jockey_place_rate', 'trainer_win_rate', 'trainer_place_rate',
    'horse_run_count'
] + [col for col in df.columns if 'horse_prev' in col]

advanced_features = [
    'gender', 'age', 'days_since_prev_race', 'weight_diff',
    'horse_jockey_place_rate', 'is_consecutive_jockey',
    'distance_diff', 'horse_venue_place_rate',
    'trainer_30d_win_rate',
    'impost_diff', 'was_accident_prev1', 'weighted_si_momentum', 'weighted_rank_momentum'
]
features = list(set(baseline_features + advanced_features))

# Preprocessing
categorical_cols = ['venue', 'state', 'gender']
for col in features:
    if col in categorical_cols:
        df[col] = df[col].astype(str).astype('category')
    else:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Split
split_date = df['date'].quantile(0.8)
train_df = df[df['date'] < split_date].copy()
test_df = df[df['date'] >= split_date].copy()

重複データを削除しました: 150000 -> 143895 件


## Optunaによるパラメータ探索

検証データに対するRMSEを最小化するパラメータを探索します。

In [3]:
def objective(trial):
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'verbosity': -1,
        'boosting_type': 'gbdt',
        'random_state': 42,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 20, 150),
        'max_depth': trial.suggest_int('max_depth', 5, 15),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.5, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
    
    # 訓練/検証
    model = lgb.LGBMRegressor(**params)
    model.fit(
        train_df[features], train_df['rank'],
        eval_set=[(test_df[features], test_df['rank'])],
        callbacks=[lgb.early_stopping(stopping_rounds=50)]
    )
    
    preds = model.predict(test_df[features])
    rmse = np.sqrt(mean_squared_error(test_df['rank'], preds))
    return rmse

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

print("Best Trial:")
print(study.best_trial.params)

[I 2026-01-27 14:37:47,477] A new study created in memory with name: no-name-3d3190e5-975b-4c8c-a80f-35a6a82f4b47


Training until validation scores don't improve for 50 rounds


[I 2026-01-27 14:37:56,495] Trial 0 finished with value: 3.3437159590180867 and parameters: {'learning_rate': 0.035093769227807346, 'num_leaves': 106, 'max_depth': 14, 'feature_fraction': 0.5612289521771443, 'bagging_fraction': 0.7653261846783888, 'bagging_freq': 4, 'min_child_samples': 26}. Best is trial 0 with value: 3.3437159590180867.


Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 3.34372
Training until validation scores don't improve for 50 rounds


[I 2026-01-27 14:38:06,850] Trial 1 finished with value: 3.448134301025562 and parameters: {'learning_rate': 0.012194132887044892, 'num_leaves': 70, 'max_depth': 8, 'feature_fraction': 0.8449744984608496, 'bagging_fraction': 0.6814986713716191, 'bagging_freq': 4, 'min_child_samples': 27}. Best is trial 0 with value: 3.3437159590180867.


...

Did not meet early stopping. Best iteration is:
[100]	valid_0's rmse: 3.33741
Training until validation scores don't improve for 50 rounds


[I 2026-01-27 14:41:14,298] Trial 28 finished with value: 3.3261896403590208 and parameters: {'learning_rate': 0.07816451685424672, 'num_leaves': 43, 'max_depth': 12, 'feature_fraction': 0.7967517388647005, 'bagging_fraction': 0.9446839107475313, 'bagging_freq': 7, 'min_child_samples': 74}. Best is trial 16 with value: 3.313837914625291.


Did not meet early stopping. Best iteration is:
[90]	valid_0's rmse: 3.32619
Training until validation scores don't improve for 50 rounds


[I 2026-01-27 14:41:24,014] Trial 29 finished with value: 3.3239078931573762 and parameters: {'learning_rate': 0.058672659486744425, 'num_leaves': 95, 'max_depth': 14, 'feature_fraction': 0.5982048785088567, 'bagging_fraction': 0.7403606453262424, 'bagging_freq': 3, 'min_child_samples': 30}. Best is trial 16 with value: 3.313837914625291.


Did not meet early stopping. Best iteration is:
[91]	valid_0's rmse: 3.32391
Best Trial:
{'learning_rate': 0.08685988063106218, 'num_leaves': 43, 'max_depth': 13, 'feature_fraction': 0.6594684801910744, 'bagging_fraction': 0.9696836279039363, 'bagging_freq': 6, 'min_child_samples': 93}


## 最良パラメータでの評価

探索で見つかった最良のパラメータを用いて最終的な精度を算出します。

In [4]:
best_params = study.best_trial.params
best_params['objective'] = 'regression'
# n_estimators は十分に大きく設定
best_model = lgb.LGBMRegressor(n_estimators=2000, **best_params)

best_model.fit(
    train_df[features], train_df['rank'],
    eval_set=[(test_df[features], test_df['rank'])],
    callbacks=[lgb.early_stopping(stopping_rounds=100)]
)

test_df['pred_score'] = best_model.predict(test_df[features])
corr, _ = spearmanr(test_df['pred_score'], test_df['rank'])
print(f"最良パラメータでの Spearman相関: {corr:.4f}")

# 的中率
test_df['pred_rank'] = test_df.groupby('race_id')['pred_score'].rank(method='min')
top1 = test_df[test_df['pred_rank'] == 1]
print(f"予測1位 勝率: {(top1['rank'] == 1).mean():.2%}")
print(f"予測1位 複勝率: {(top1['rank'] <= 3).mean():.2%}")

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[104]	valid_0's l2: 10.9899
最良パラメータでの Spearman相関: 0.4829
予測1位 勝率: 24.09%
予測1位 複勝率: 61.55%
